# Exploração Inicial do Dataset Fashion MNIST

Neste notebook, vamos explorar o dataset Fashion MNIST e testar nossa implementação de MLP. Primeiro vamos carregar as bibliotecas necessárias e os dados.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from example import MLP

In [ ]:
# Carregando os dados
train = pd.read_csv('data/fashion-mnist_train.csv')
test = pd.read_csv('data/fashion-mnist_test.csv')

print(f"Formato do conjunto de treinamento: {train.shape}")
print(f"Formato do conjunto de teste: {test.shape}")

In [ ]:
train.head()

In [ ]:
# Distribuição das classes
plt.figure(figsize=(10, 6))
sns.countplot(x='label', data=train)
plt.title('Distribuição das Classes no Conjunto de Treinamento')
plt.xlabel('Classe')
plt.ylabel('Contagem')

# Mapear rótulos para nomes das classes
class_names = ['Camiseta', 'Calça', 'Pulôver', 'Vestido', 'Casaco', 
              'Sandália', 'Camisa', 'Tênis', 'Bolsa', 'Bota']
plt.xticks(range(10), class_names, rotation=45)
plt.show()

## Visualização de Exemplos

In [ ]:
def plot_examples(data, labels, num_examples=5):
    plt.figure(figsize=(15, 3))
    for i in range(num_examples):
        plt.subplot(1, num_examples, i+1)
        # Converter vetor para matriz 28x28
        img = data.iloc[i, 1:].values.reshape(28, 28)
        plt.imshow(img, cmap='gray')
        plt.title(class_names[labels.iloc[i]])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Mostrar 5 exemplos aleatórios
random_indices = np.random.randint(0, len(train), 5)
plot_examples(train.iloc[random_indices], train.iloc[random_indices]['label'])

## Preparação dos Dados

Vamos preparar os dados para treinamento:

In [ ]:
# Separar features e labels
X_train = train.drop('label', axis=1).values / 255.0  # Normalizar para [0, 1]
y_train = train['label'].values

X_test = test.drop('label', axis=1).values / 255.0
y_test = test['label'].values

# Converter para one-hot encoding
def to_onehot(y, num_classes=10):
    onehot = np.zeros((len(y), num_classes))
    onehot[np.arange(len(y)), y] = 1
    return onehot

y_train_onehot = to_onehot(y_train)
y_test_onehot = to_onehot(y_test)

# Separar um conjunto de validação
X_train, X_val, y_train_onehot, y_val_onehot = train_test_split(
    X_train, y_train_onehot, test_size=0.1, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train_onehot.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

## Treinamento da MLP

Agora vamos treinar um modelo MLP usando nossa implementação personalizada:

In [ ]:
# Criando uma instância da MLP
mlp = MLP(
    input_size=784,          # 28x28 pixels
    hidden_size=128,         # Número de neurônios na camada oculta
    output_size=10,          # 10 classes
    activation='sigmoid',    # Função de ativação
    learning_rate=0.01,      # Taxa de aprendizado
    momentum=0.9,            # Termo de momentum
    weight_init='xavier',    # Inicialização de pesos
    random_state=42          # Para reprodutibilidade
)

In [ ]:
# Treinar o modelo
history = mlp.train(
    X_train,
    y_train_onehot,
    epochs=10,                  # Número de épocas
    batch_size=64,              # Tamanho do mini-batch
    validation_data=(X_val, y_val_onehot),
    verbose=True,
    log_interval=1              # Mostrar progresso a cada época
)

In [ ]:
# Plotar curvas de aprendizado
plt.figure(figsize=(12, 5))

# Perda
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Treinamento')
plt.plot(history['val_loss'], label='Validação')
plt.title('Curva de Perda')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

# Acurácia
plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='Treinamento')
plt.plot(history['val_accuracy'], label='Validação')
plt.title('Curva de Acurácia')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.tight_layout()
plt.show()

## Avaliação no Conjunto de Teste

In [ ]:
# Avaliar no conjunto de teste
test_loss, test_acc = mlp.evaluate(X_test, y_test_onehot)
print(f"Perda no teste: {test_loss:.4f}")
print(f"Acurácia no teste: {test_acc:.4f}")

In [ ]:
# Predições no conjunto de teste
y_pred = mlp.predict_classes(X_test)

# Matriz de confusão
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Matriz de Confusão')
plt.xlabel('Predição')
plt.ylabel('Verdadeiro')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=class_names))

## Salvando e Carregando o Modelo

In [ ]:
# Salvar o modelo
model_path = 'fashion_mnist_model.pkl'
mlp.save(model_path)
print(f"Modelo salvo em {model_path}")

In [ ]:
# Carregar o modelo
loaded_mlp = MLP.load(model_path)
print("Modelo carregado com sucesso!")

# Verificar se o modelo carregado tem o mesmo desempenho
loaded_loss, loaded_acc = loaded_mlp.evaluate(X_test, y_test_onehot)
print(f"Perda no teste (modelo carregado): {loaded_loss:.4f}")
print(f"Acurácia no teste (modelo carregado): {loaded_acc:.4f}")

## Visualização de Exemplos Preditos

In [ ]:
# Visualizar algumas predições
def plot_predictions(X, y_true, y_pred, num_examples=5):
    indices = np.random.choice(len(X), num_examples, replace=False)

    plt.figure(figsize=(15, 3))
    for i, idx in enumerate(indices):
        plt.subplot(1, num_examples, i+1)
        img = X[idx].reshape(28, 28)
        plt.imshow(img, cmap='gray')

        pred_class = class_names[y_pred[idx]]
        true_class = class_names[y_true[idx]]

        color = 'green' if pred_class == true_class else 'red'
        plt.title(f"Pred: {pred_class}\nTrue: {true_class}", color=color)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Mostrar exemplos com suas predições
plot_predictions(X_test, y_test, y_pred, num_examples=8)

## Experimentos com Hiperparâmetros

Podemos testar diferentes configurações para otimizar o desempenho:

In [ ]:
# Função para treinar e avaliar um modelo com configurações específicas
def experiment(hidden_size, activation, learning_rate, momentum, weight_init, epochs=5):
    print(f"\nExperimento: hidden_size={hidden_size}, activation={activation}, "
          f"lr={learning_rate}, momentum={momentum}, init={weight_init}")

    model = MLP(
        input_size=784,
        hidden_size=hidden_size,
        output_size=10,
        activation=activation,
        learning_rate=learning_rate,
        momentum=momentum,
        weight_init=weight_init,
        random_state=42
    )

    # Treinar com um subset para economizar tempo
    subset_size = 10000
    X_subset = X_train[:subset_size]
    y_subset = y_train_onehot[:subset_size]

    history = model.train(
        X_subset,
        y_subset,
        epochs=epochs,
        batch_size=64,
        validation_data=(X_val, y_val_onehot),
        verbose=True
    )

    test_loss, test_acc = model.evaluate(X_test, y_test_onehot)
    print(f"Acurácia no teste: {test_acc:.4f}")

    return model, history, test_acc

In [ ]:
# Experimento 1: ReLU com inicialização He
model1, history1, acc1 = experiment(
    hidden_size=128, 
    activation='relu', 
    learning_rate=0.001, 
    momentum=0.9, 
    weight_init='he'
)

In [ ]:
# Experimento 2: Tanh com inicialização Xavier
model2, history2, acc2 = experiment(
    hidden_size=128, 
    activation='tanh', 
    learning_rate=0.001, 
    momentum=0.9, 
    weight_init='xavier'
)

In [ ]:
# Experimento 3: Sigmoid com taxa de aprendizado maior
model3, history3, acc3 = experiment(
    hidden_size=128, 
    activation='sigmoid', 
    learning_rate=0.01, 
    momentum=0.9, 
    weight_init='xavier'
)

In [ ]:
# Comparar resultados
results = {
    'ReLU + He': acc1,
    'Tanh + Xavier': acc2,
    'Sigmoid + Xavier': acc3
}

plt.figure(figsize=(10, 6))
plt.bar(results.keys(), results.values())
plt.ylim(0.7, 1.0)  # Ajustar para melhor visualização
plt.title('Comparação de Acurácia por Configuração')
plt.ylabel('Acurácia no Teste')
for i, (k, v) in enumerate(results.items()):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.show()

## Conclusão

Neste notebook, exploramos o dataset Fashion MNIST e implementamos uma MLP com diversos parâmetros configuráveis. Os principais pontos observados foram:

1. A inicialização de pesos adequada para cada função de ativação é crucial
2. O uso de momentum melhora a convergência do treinamento
3. Diferentes funções de ativação têm desempenhos distintos neste problema

Para melhorar ainda mais o desempenho, poderíamos:
- Adicionar mais camadas ocultas
- Implementar técnicas como dropout e batch normalization
- Explorar outras técnicas de otimização como Adam, RMSProp
- Realizar uma busca mais ampla de hiperparâmetros